In [1]:
'''!pip install aicrowd-cli
API_KEY = "9967b67c796b4d50e5d26f4dfd97ed06" 
!aicrowd login --api-key $API_KEY
!aicrowd dataset download --challenge global-wheat-challenge-2021'''

'!pip install aicrowd-cli\nAPI_KEY = "9967b67c796b4d50e5d26f4dfd97ed06" \n!aicrowd login --api-key $API_KEY\n!aicrowd dataset download --challenge global-wheat-challenge-2021'

In [2]:
'''import shutil
shutil.copyfile('/content/train.zip',
                '/content/drive/MyDrive/aicrowd_wheat/train.zip')
shutil.copyfile('/content/test.zip',
                '/content/drive/MyDrive/aicrowd_wheat/test.zip')
shutil.copyfile('/content/submission.csv',
                '/content/drive/MyDrive/aicrowd_wheat/submission.csv')'''

"import shutil\nshutil.copyfile('/content/train.zip',\n                '/content/drive/MyDrive/aicrowd_wheat/train.zip')\nshutil.copyfile('/content/test.zip',\n                '/content/drive/MyDrive/aicrowd_wheat/test.zip')\nshutil.copyfile('/content/submission.csv',\n                '/content/drive/MyDrive/aicrowd_wheat/submission.csv')"

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jun 16 06:39:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -q albumentations==0.4.6
!pip install mlflow

     |████████████████████████████████| 122kB 7.0MB/s 
     |████████████████████████████████| 952kB 11.5MB/s 
     |████████████████████████████████| 14.2MB 7.4MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 153kB 58.4MB/s 
     |████████████████████████████████| 163kB 46.5MB/s 
     |████████████████████████████████| 1.1MB 43.6MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.14.3-cp37-none-any.whl size=100560 sha256=ecff97affbf79c85d337ab63c4a02607af5c50ba95c0218bc3bfd9088f2babbe
  Stored in directory: /root/.cache/pip/wheels/5b/24/f3/34d8e3964dac4ba849d844273c49a679111b00d5799ebb934a
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.2-cp37-none-any.whl size=174

In [6]:
import shutil
import pandas as pd
import numpy as np 
import os
import cv2
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN  
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
from datetime import datetime

from sklearn.model_selection import StratifiedKFold, train_test_split
import joblib
import mlflow

In [7]:
from torchvision.models.detection._utils import Matcher
from torchvision.ops.boxes import box_iou

def accuracy(src_boxes, pred_boxes,  iou_threshold = 1.):
  """
  The accuracy method is not the one used in the evaluator but very similar
  """
  total_gt = len(src_boxes)
  total_pred = len(pred_boxes)
  if total_gt > 0 and total_pred > 0:


    # Define the matcher and distance matrix based on iou
    matcher = Matcher(iou_threshold,iou_threshold,allow_low_quality_matches=False) 
    match_quality_matrix = box_iou(src_boxes,pred_boxes)

    results = matcher(match_quality_matrix)
    
    true_positive = torch.count_nonzero(results.unique() != -1)
    matched_elements = results[results > -1]
    
    #in Matcher, a pred element can be matched only twice 
    false_positive = torch.count_nonzero(results == -1) + ( len(matched_elements) - len(matched_elements.unique()))
    false_negative = total_gt - true_positive

        
    return  true_positive / ( true_positive + false_positive + false_negative )

  elif total_gt == 0:
      if total_pred > 0:
          return torch.tensor(0.).cuda()
      else:
          return torch.tensor(1.).cuda()
  elif total_gt > 0 and total_pred == 0:
      return torch.tensor(0.).cuda()

In [8]:
if(os.path.isdir('/content/drive/MyDrive/aicrowd_wheat/mlruns/')):
  shutil.copytree('/content/drive/MyDrive/aicrowd_wheat/mlruns/','/content/mlruns/')

In [9]:
if(os.path.isdir('train')):
  print('folder exists, skip unzip procedure')
else:
  shutil.copyfile('/content/drive/MyDrive/aicrowd_wheat/train.zip','/content/train.zip')
  shutil.unpack_archive('./train.zip')
  #!unzip train.zip

In [10]:
len(os.listdir('./train/')), os.listdir('./train/')[0:3]

(3655,
 ['424ae806fa5cd41a39d55fcf711b18c5e928bf1b501c0cc6e1b5f4628d83011f.png',
  'e38255a86b481f787da4b7ae4a227f6ded7c23b36438af45e63ed67f2dfc4fb4.png',
  '27f320c0ead2046746c228d82001ab73495cc161d57c2f6d4046f9c89d045cf2.png'])

In [11]:
df_train = pd.read_csv('./train.csv')
df_train['image_id'] = df_train['image_name']
df_train['has_no_boxes'] = np.where(df_train['BoxesString']=='no_box',1,0)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657 entries, 0 to 3656
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_name    3657 non-null   object
 1   BoxesString   3657 non-null   object
 2   domain        3657 non-null   int64 
 3   image_id      3657 non-null   object
 4   has_no_boxes  3657 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 143.0+ KB


In [12]:
df_train['has_no_boxes'].sum()*100/df_train.shape[0]

1.3672409078479628

In [13]:
df_noboxes = df_train.groupby(['domain'])['has_no_boxes'].agg(['sum','count']).reset_index()
df_noboxes['perc'] = df_noboxes['sum']*100/df_noboxes['count']
df_noboxes

,domain,sum,count,perc
0,0,0,29,0.000000
1,1,2,60,3.333333
2,2,0,176,0.000000
3,3,0,20,0.000000
4,4,0,24,0.000000
5,5,46,448,10.267857
6,6,0,160,0.000000
7,7,0,60,0.000000
8,8,0,32,0.000000
9,9,0,82,0.000000


In [14]:
df_train[df_train.BoxesString=='no_box'].shape[0], \
df_train[df_train.BoxesString=='no_box'].shape[0]*100/df_train.shape[0]

(50, 1.3672409078479628)

### Filter images with multiple domains

In [15]:
tf = df_train[['image_id','domain']].groupby(['image_id']).count().reset_index()
domain_lkup = df_train[~df_train.image_id.isin(tf[tf.domain>1].image_id.values)][['image_id','domain']]
tf[tf.domain>1].image_id.values

array(['73d4c61b8be64a946df940e3e5cdceecb662960363c84a29fb8e99d4b660fa0e',
       '8ec1474b9cdc0fd870b443439daa56660c57140c051a8a49052d22d828728f12'],
      dtype=object)

In [16]:
df_train_final = pd.concat([pd.Series(row['image_id'], row['BoxesString'].split(';')) \
                            for _, row in df_train[df_train.BoxesString!='no_box'].iterrows()]).reset_index()
df_train_final.columns = ['box','image_id']
df_train_final['x_min'] = df_train_final.box.apply(lambda x:x.split(' ')[0]).apply(int)
df_train_final['y_min'] = df_train_final.box.apply(lambda x:x.split(' ')[1]).apply(int)
df_train_final['x_max'] = df_train_final.box.apply(lambda x:x.split(' ')[2]).apply(int)
df_train_final['y_max'] = df_train_final.box.apply(lambda x:x.split(' ')[3]).apply(int)
df_train_final['x'] = df_train_final['x_min']
df_train_final['y'] = df_train_final['y_min']
df_train_final['w'] = df_train_final['x_max'] - df_train_final['x_min']
df_train_final['h'] = df_train_final['y_max'] - df_train_final['y_min']
df_train_final = df_train_final.merge(domain_lkup, on=['image_id'], how='left')
df_train_final.fillna(-1, inplace=True)
df_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163690 entries, 0 to 163689
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   box       163690 non-null  object 
 1   image_id  163690 non-null  object 
 2   x_min     163690 non-null  int64  
 3   y_min     163690 non-null  int64  
 4   x_max     163690 non-null  int64  
 5   y_max     163690 non-null  int64  
 6   x         163690 non-null  int64  
 7   y         163690 non-null  int64  
 8   w         163690 non-null  int64  
 9   h         163690 non-null  int64  
 10  domain    163690 non-null  float64
dtypes: float64(1), int64(8), object(2)
memory usage: 15.0+ MB


In [17]:
df_train_final.head()
# df_train.domain.value_counts()

,box,image_id,x_min,y_min,x_max,y_max,x,y,w,h,domain
0,949 967 999 994,7b73239dfd89b06c03e1be81cc5074ec47ae048305ec63...,949,967,999,994,949,967,50,27,0.0
1,368 649 443 685,7b73239dfd89b06c03e1be81cc5074ec47ae048305ec63...,368,649,443,685,368,649,75,36,0.0
2,118 280 185 333,7b73239dfd89b06c03e1be81cc5074ec47ae048305ec63...,118,280,185,333,118,280,67,53,0.0
3,899 564 947 609,7b73239dfd89b06c03e1be81cc5074ec47ae048305ec63...,899,564,947,609,899,564,48,45,0.0
4,604 243 682 297,7b73239dfd89b06c03e1be81cc5074ec47ae048305ec63...,604,243,682,297,604,243,78,54,0.0


In [18]:
DIR_INPUT = './'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

In [19]:
'''train_set, test_set = train_test_split(df_train.image_id.values, test_size=0.2, random_state=42)
train_set.shape, test_set.shape
joblib.dump(train_set, '/content/drive/MyDrive/aicrowd_wheat/train_set')
joblib.dump(test_set, '/content/drive/MyDrive/aicrowd_wheat/test_set')
kfold = StratifiedKFold(n_splits=5)
folds = []
for train_ids, test_ids in kfold.split(df_train.image_id.values, df_train.domain.values):
  folds.append([df_train.image_id.values[train_ids], df_train.image_id.values[test_ids]])
joblib.dump(folds, '/content/drive/MyDrive/aicrowd_wheat/kfolds')'''

"train_set, test_set = train_test_split(df_train.image_id.values, test_size=0.2, random_state=42)\ntrain_set.shape, test_set.shape\njoblib.dump(train_set, '/content/drive/MyDrive/aicrowd_wheat/train_set')\njoblib.dump(test_set, '/content/drive/MyDrive/aicrowd_wheat/test_set')\nkfold = StratifiedKFold(n_splits=5)\nfolds = []\nfor train_ids, test_ids in kfold.split(df_train.image_id.values, df_train.domain.values):\n  folds.append([df_train.image_id.values[train_ids], df_train.image_id.values[test_ids]])\njoblib.dump(folds, '/content/drive/MyDrive/aicrowd_wheat/kfolds')"

In [20]:
train_images = joblib.load('/content/drive/MyDrive/aicrowd_wheat/train_set')
test_images = joblib.load('/content/drive/MyDrive/aicrowd_wheat/test_set')
kfold_images = joblib.load('/content/drive/MyDrive/aicrowd_wheat/kfolds')
train_images.shape, test_images.shape, len(kfold_images)

((2925,), (732,), 5)

In [21]:
valid_df = df_train_final[df_train_final['image_id'].isin(test_images)]
train_df = df_train_final[df_train_final['image_id'].isin(train_images)]
valid_df.shape, train_df.shape

((32281, 11), (131476, 11))

In [22]:
class WheatDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}.png', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [23]:
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        # A.ShiftScaleRotate(p=0.5),
        # A.RandomBrightnessContrast(p=0.3),
        # A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=0.3),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def collate_fn(batch):
    return tuple(zip(*batch))

## Train Model

In [24]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
#model.load_state_dict(torch.load('/content/drive/MyDrive/aicrowd_wheat/yolov5/'\
#                                 +'kaggle_best.pt'))

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


ModuleNotFoundError: ignored

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

### Full dataset train

In [ ]:
# train_dataset = WheatDataset(train_df, DIR_TRAIN, get_train_transform())
train_dataset = WheatDataset(df_train_final, DIR_TRAIN, get_train_transform())
valid_dataset = WheatDataset(valid_df, DIR_TRAIN, get_valid_transform())

# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
boxes = targets[1]['boxes'].cpu().numpy().astype(np.int32)
sample = images[1].permute(1,2,0).cpu().numpy()
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
lr_rate=0.005
optimizer = torch.optim.SGD(params, lr=lr_rate, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
# lr_scheduler = None

num_epochs = 40

In [ ]:
loss_hist = Averager()
itr = 1
loss_plot = []

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")  
    loss_plot.append(loss_hist.value)

In [ ]:
plt.plot(loss_plot)

In [ ]:
filename = 'fasterrcnn_resnet50_epochs_'+str(num_epochs)+'_lr_'+str(lr_rate)+'_time_'+datetime.now().strftime(format='%d_%m_%Y_%H_%M')+'.pth'
torch.save(model.state_dict(), '/content/drive/MyDrive/aicrowd_wheat/'+filename)

## Evaluation

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
valid_dataset = WheatDataset(valid_df, DIR_TRAIN, get_valid_transform())
valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn,
    drop_last=False
)

num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load('/content/drive/MyDrive/aicrowd_wheat/'\
                                 +'fasterrcnn_resnet50_epochs_40_lr_0.005_time_13_06_2021_21_23.pth'))
model.eval()
x = model.to(device)

In [ ]:
device

In [ ]:
results = []

for images, targets, image_ids in valid_data_loader:
  images = list(img.to(device) for img in images)
  outputs = model(images)
  # outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
  results.append([image_ids[0], targets[0], 
                  outputs[0]['boxes'].cpu().detach().numpy(), outputs[0]['scores'].cpu().detach().numpy()])

len(results)

In [ ]:
threshold = 0.6
performance = []
for row in results:
  image_id = row[0]
  actuals = row[1]['boxes']#.cpu().numpy().astype(np.int32)
  scores = row[3]
  predictions = row[2][scores>=threshold]
  acc = accuracy(torch.from_numpy(predictions), actuals, iou_threshold = .5)
  acc = np.float(acc.detach().cpu().numpy())
  performance.append([image_id, acc])
  
print(len(performance))
df_performance = pd.DataFrame(performance, columns=['image_id','acc'])
df_performance.acc.mean()

#### 20 Epochs 0.7653925410826892

In [ ]:
for threshold in np.arange(0,1,0.05):
  performance = []
  for row in results:
    image_id = row[0]
    actuals = row[1]['boxes']#.cpu().numpy().astype(np.int32)
    scores = row[3]
    predictions = row[2][scores>=threshold]
    acc = accuracy(torch.from_numpy(predictions), actuals, iou_threshold = .5)
    acc = np.float(acc.detach().cpu().numpy())
    performance.append([image_id, acc])
    df_performance = pd.DataFrame(performance, columns=['image_id','acc'])
    acc = df_performance.acc.mean()
    #mlflow.log_param("threshold", threshold)
    #mlflow.log_param("num_epochs", num_epochs)
    #mlflow.log_param("lr_rate", lr_rate)
    #mlflow.log_metric("accuracy", acc)
  print('threshold:',round(threshold,2),', acc:',round(acc,4))

In [ ]:
# shutil.copytree('/content/mlruns/','/content/drive/MyDrive/aicrowd_wheat/mlruns/')

In [ ]:
images, targets, image_ids = next(iter(valid_data_loader))
images = list(img.to(device) for img in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()
model.eval()
cpu_device = torch.device("cpu")
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]


fig, ax = plt.subplots(1, 1, figsize=(16, 8))
boxes = outputs[0]['boxes'].detach().numpy()
scores = outputs[0]['scores'].detach().numpy()
boxes = boxes[scores>=0.5]
for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
shutil.copyfile('/content/train.csv',
                '/content/drive/MyDrive/aicrowd_wheat/train.csv')

In [ ]:
while True:
  pass